### Simulatore dati e streaming verso Event Hub
v2.0

In [2]:
import numpy as np
import json
import time
import datetime
import uuid

In [3]:
!pip install azure.eventhub

     |████████████████████████████████| 61kB 2.7MB/s eta 0:00:011
     |████████████████████████████████| 2.8MB 9.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from azure.eventhub import EventHubClient, Sender, EventData

Some notes:

- The batches are defined below in Batches definition (they could be generated in a loop by indicanting just the number of batches)
- All batches involve the same machine/sensors (this can be easily modified by generating the random GUID's inside the loop)
- The duration of the simulation is defined by batch_planning and batch_duration

In [5]:
# Function temperature sensor
def sensor_temp(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = {'batch-id': batch['ID_prefix'] + datetimeID,
            'channel-id': id,
            'val': y,
            'time': timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            }
    return data

# Function pressure sensor
def sensor_press(min, max, id, batch, datetimeID):
    y = round(np.random.uniform(min, max), 2)
    timestamp = datetime.datetime.now()
    data = {'batch-id': batch['ID_prefix'] + datetimeID,
            'channel-id': id,
            'val': y,
            'time': timestamp.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            }
    return data

In [25]:
# Batches definition
batches = {1: {'batch_description': 'injection piece A1', 'ID_prefix': 'INJPCA1_'},
                2: {'batch_description': 'injection piece A2', 'ID_prefix': 'INJPCA2_'},
                3: {'batch_description': 'injection piece A3', 'ID_prefix': 'INJPCA3_'},
                4: {'batch_description': 'injection piece A4', 'ID_prefix': 'INJPCA4_'},
                5: {'batch_description': 'injection piece A5', 'ID_prefix': 'INJPCA5_'}}

# Batches scheduling
batch_planning = [1, 5] #[1, 2, 3, 1, 2, 3, 1, 1] # sequence of batches = keys in batches dictionary
N_batches = len(batch_planning) # total number of batches

# Batches duration (choose batch_duration = batch_duration_fix OR batch_duration = batch_duration_random)
minutes_min, minutes_max = 1, 3 # range of batch duration in minutes (for random)
minutes_fix = 1 # batch duration in minutes (for fix)
batch_duration_fix = [1]*N_batches
batch_duration_fix = [batch_duration_fix[i]*minutes_fix for i in batch_duration_fix]
batch_duration_random = np.random.randint(minutes_min, minutes_max + 1, N_batches).tolist()
batch_duration = batch_duration_fix #batch_duration_random #batch_duration_fix

# ID producers: list of predefined ID's or randomly generated GUID's
producer_ID = ['T1', 'P1', 'T2', 'P2', 'T3', 'P3'] # list of predefined ID's
N_channels = len(producer_ID)

if producer_ID == []:
    # ID producers (sensor channel identifiers, 36-digits randomly generated GUID according to standard RFC-4122)
    N_channels = 4
    producer_ID = [str(uuid.uuid4()) for i in range(N_channels)] # list of ID's, example 'f66aedac-9749-4491-837a-f0cb623cc2df'

# Sampling frequency (unique for all sensors)
sampling_freq = 2 # measurements/sec

In [6]:
# INFORMATION ABOUT THE CREATED EVENT HUBS NAMESPACE AND EVENT HUB
'''
ADDRESS = 'xxxxx' # example 'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1'
USER = 'RootManageSharedAccessKey'
KEY = 'xxxxx' # Primary key
client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
sender = client.add_sender(partition="0")
client.run()
'''

'\nADDRESS = \'xxxxx\' # example \'amqps://eventhub1-idaq-test6.servicebus.windows.net/eventhub1\'\nUSER = \'RootManageSharedAccessKey\'\nKEY = \'xxxxx\' # Primary key\nclient = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)\nsender = client.add_sender(partition="0")\nclient.run()\n'

In [26]:
# Loop generating measurements
start_time = time.time()
datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

batch_order = 0 # element of batch_planning list

while(batch_order < N_batches):

    N_batch = batch_planning[batch_order]
    
    for producer in zip(producer_ID[::2], producer_ID[1::2]):
        
        # First sensor data
        data = sensor_temp(20, 30, producer[0], batches[N_batch], datetimeID) # specify range for measured value
        datax = json.dumps(data)
        print(datax)
        #sender.send(EventData(datax))
        
        # Second sensor data
        data = sensor_press(120, 160, producer[1], batches[N_batch], datetimeID) # specify range for measured value
        datax = json.dumps(data)
        print(datax)
        #sender.send(EventData(datax))
        
        # Add more sensors here, maximum number of sensors = N_channels
    
    time.sleep(1 / sampling_freq)

    elapsed_time = (time.time() - start_time) / 60 # minutes
    
    if elapsed_time > batch_duration[batch_order]: # in minutes
        batch_order += 1
        start_time = time.time()
        datetimeID = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M")

#client.stop()

{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 27.75, "time": "2019-05-10 12:19:14.238"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 130.32, "time": "2019-05-10 12:19:14.238"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 26.55, "time": "2019-05-10 12:19:14.238"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P2", "val": 142.53, "time": "2019-05-10 12:19:14.240"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T3", "val": 25.05, "time": "2019-05-10 12:19:14.240"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P3", "val": 129.28, "time": "2019-05-10 12:19:14.240"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 21.2, "time": "2019-05-10 12:19:14.741"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 142.83, "time": "2019-05-10 12:19:14.741"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 20.76, "time": "2019-05-10 12:19:14.743"}
{"batch

{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 23.48, "time": "2019-05-10 12:19:20.773"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 137.26, "time": "2019-05-10 12:19:20.773"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 28.44, "time": "2019-05-10 12:19:20.773"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P2", "val": 140.95, "time": "2019-05-10 12:19:20.773"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T3", "val": 22.39, "time": "2019-05-10 12:19:20.774"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P3", "val": 127.88, "time": "2019-05-10 12:19:20.774"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 23.27, "time": "2019-05-10 12:19:21.274"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 133.72, "time": "2019-05-10 12:19:21.275"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 24.32, "time": "2019-05-10 12:19:21.275"}
{"batc

{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 27.36, "time": "2019-05-10 12:19:27.298"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 153.16, "time": "2019-05-10 12:19:27.298"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 20.7, "time": "2019-05-10 12:19:27.299"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P2", "val": 126.07, "time": "2019-05-10 12:19:27.299"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T3", "val": 22.22, "time": "2019-05-10 12:19:27.299"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P3", "val": 141.68, "time": "2019-05-10 12:19:27.299"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 27.96, "time": "2019-05-10 12:19:27.800"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 154.43, "time": "2019-05-10 12:19:27.800"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 29.81, "time": "2019-05-10 12:19:27.800"}
{"batch

{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 21.1, "time": "2019-05-10 12:19:33.827"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 147.18, "time": "2019-05-10 12:19:33.827"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 26.6, "time": "2019-05-10 12:19:33.827"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P2", "val": 133.45, "time": "2019-05-10 12:19:33.828"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T3", "val": 20.04, "time": "2019-05-10 12:19:33.828"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P3", "val": 146.99, "time": "2019-05-10 12:19:33.828"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 20.89, "time": "2019-05-10 12:19:34.329"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 122.21, "time": "2019-05-10 12:19:34.330"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 27.62, "time": "2019-05-10 12:19:34.330"}
{"batch-

{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 26.58, "time": "2019-05-10 12:19:40.352"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 158.47, "time": "2019-05-10 12:19:40.353"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 23.74, "time": "2019-05-10 12:19:40.353"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P2", "val": 137.4, "time": "2019-05-10 12:19:40.353"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T3", "val": 23.09, "time": "2019-05-10 12:19:40.353"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P3", "val": 128.57, "time": "2019-05-10 12:19:40.353"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T1", "val": 23.04, "time": "2019-05-10 12:19:40.854"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "P1", "val": 141.13, "time": "2019-05-10 12:19:40.854"}
{"batch-id": "INJPCA1_2019_05_10_12_19", "channel-id": "T2", "val": 23.86, "time": "2019-05-10 12:19:40.854"}
{"batch

...

{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 20.9, "time": "2019-05-10 12:20:45.616"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 131.5, "time": "2019-05-10 12:20:45.617"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 26.85, "time": "2019-05-10 12:20:45.617"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P2", "val": 158.65, "time": "2019-05-10 12:20:45.617"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T3", "val": 20.3, "time": "2019-05-10 12:20:45.617"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P3", "val": 155.93, "time": "2019-05-10 12:20:45.617"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 21.15, "time": "2019-05-10 12:20:46.118"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 125.25, "time": "2019-05-10 12:20:46.118"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 25.01, "time": "2019-05-10 12:20:46.118"}
{"batch-i

{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 21.73, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 124.66, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 22.4, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P2", "val": 147.74, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T3", "val": 22.09, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P3", "val": 144.97, "time": "2019-05-10 12:20:52.145"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 21.86, "time": "2019-05-10 12:20:52.646"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 131.35, "time": "2019-05-10 12:20:52.646"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 26.76, "time": "2019-05-10 12:20:52.646"}
{"batch

{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 24.52, "time": "2019-05-10 12:20:58.672"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 148.35, "time": "2019-05-10 12:20:58.672"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 23.85, "time": "2019-05-10 12:20:58.673"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P2", "val": 150.21, "time": "2019-05-10 12:20:58.673"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T3", "val": 24.64, "time": "2019-05-10 12:20:58.673"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P3", "val": 157.18, "time": "2019-05-10 12:20:58.673"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 22.6, "time": "2019-05-10 12:20:59.174"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 125.45, "time": "2019-05-10 12:20:59.174"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 22.59, "time": "2019-05-10 12:20:59.174"}
{"batch

{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 28.62, "time": "2019-05-10 12:21:05.199"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 152.25, "time": "2019-05-10 12:21:05.199"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 21.6, "time": "2019-05-10 12:21:05.200"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P2", "val": 149.12, "time": "2019-05-10 12:21:05.200"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T3", "val": 26.62, "time": "2019-05-10 12:21:05.200"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P3", "val": 148.88, "time": "2019-05-10 12:21:05.200"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 25.01, "time": "2019-05-10 12:21:05.701"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 129.82, "time": "2019-05-10 12:21:05.702"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 26.33, "time": "2019-05-10 12:21:05.702"}
{"batch

{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 21.31, "time": "2019-05-10 12:21:11.727"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 127.88, "time": "2019-05-10 12:21:11.727"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 30.0, "time": "2019-05-10 12:21:11.727"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P2", "val": 149.83, "time": "2019-05-10 12:21:11.728"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T3", "val": 22.87, "time": "2019-05-10 12:21:11.728"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P3", "val": 124.18, "time": "2019-05-10 12:21:11.728"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T1", "val": 28.08, "time": "2019-05-10 12:21:12.228"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "P1", "val": 133.92, "time": "2019-05-10 12:21:12.229"}
{"batch-id": "INJPCA5_2019_05_10_12_20", "channel-id": "T2", "val": 28.24, "time": "2019-05-10 12:21:12.229"}
{"batch